## 세마포어 Semaphore

- 가용한 자원의 수를 가지고 있는 객체
- 세마포어에 있는 자원 = 공유 변수
- 세마포어의 공유 변수가 없다면 다음 프로세스는 wait 또는 sleep
- p 연산 = 공유 변수를 감소하는 연산, v 연산 = 공유 변수를 증가시키는 연산

In [5]:
import threading
import time

class ThreadPool(object):
    def __init__(self):
        self.active = []
        self.lock = threading.Lock()
        
    def acquire(self, name):
        with self.lock: # 맨 처음 진입한 것에 대한 lock 획득
            self.active.append(name)
            print("획득 : {0} | 스레드 풀 : {1}".format(name, self.active))
            
    def release(self, name): #lock을 풀 때 지원
        with self.lock:
            self.active.remove(name)
            print("반환 : {0} | 스레드 풀 : {1}".format(name, self.active))
            
def worker(semaphore, pool):
    with semaphore:
        name = threading.currentThread().getName()
        pool.acquire(name)
        time.sleep(1)
        pool.release(name)

if __name__ == "__main__":
    threads = []
    pool = ThreadPool()
    semaphore = threading.Semaphore(3)
    for i in range(10):
        t = threading.Thread(target = worker, name = "스레드" + str(i), args = (semaphore, pool))
        t.start()
        threads.append(t)
        
    for t in threads:
        t.join()

획득 : 스레드0 | 스레드 풀 : ['스레드0']
획득 : 스레드1 | 스레드 풀 : ['스레드0', '스레드1']
획득 : 스레드2 | 스레드 풀 : ['스레드0', '스레드1', '스레드2']
반환 : 스레드0 | 스레드 풀 : ['스레드1', '스레드2']
획득 : 스레드3 | 스레드 풀 : ['스레드1', '스레드2', '스레드3']
반환 : 스레드1 | 스레드 풀 : ['스레드2', '스레드3']
획득 : 스레드4 | 스레드 풀 : ['스레드2', '스레드3', '스레드4']
반환 : 스레드2 | 스레드 풀 : ['스레드3', '스레드4']
획득 : 스레드5 | 스레드 풀 : ['스레드3', '스레드4', '스레드5']
반환 : 스레드3 | 스레드 풀 : ['스레드4', '스레드5']
획득 : 스레드6 | 스레드 풀 : ['스레드4', '스레드5', '스레드6']
반환 : 스레드4 | 스레드 풀 : ['스레드5', '스레드6']
획득 : 스레드7 | 스레드 풀 : ['스레드5', '스레드6', '스레드7']
반환 : 스레드5 | 스레드 풀 : ['스레드6', '스레드7']
획득 : 스레드8 | 스레드 풀 : ['스레드6', '스레드7', '스레드8']
반환 : 스레드6 | 스레드 풀 : ['스레드7', '스레드8']
획득 : 스레드9 | 스레드 풀 : ['스레드7', '스레드8', '스레드9']
반환 : 스레드7 | 스레드 풀 : ['스레드8', '스레드9']
반환 : 스레드8 | 스레드 풀 : ['스레드9']
반환 : 스레드9 | 스레드 풀 : []


## 데드락과 스핀락 

### 생성자, 소비자의 문제 

In [15]:
import threading

def consumer(cond):
    name = threading.currentThread().getName()
    print("{0} 시작".format(name))
    with cond:
        print("{0} 대기".format(name))
        cond.wait()
        print("{0} 자원 소비".format(name))
        
def producer(cond):
    name = threading.currentThread().getName()
    print("{0} 시작".format(name))
    with cond:
        print("{0} 자원 생산 후 모든 소비자에게 알림".format(name))
        cond.notifyAll()
        
if __name__ == "__main__":
    condition = threading.Condition()
    consumer1 = threading.Thread(name = "소비자 1", target = consumer, args = (condition,))
    consumer2 = threading.Thread(name = "소비자 2", target = consumer, args = (condition,))
    producer = threading.Thread(name = "생산자", target = producer, args = (condition,))
    
    consumer1.start()
    consumer2.start()
    producer.start()

소비자 1 시작
소비자 1 대기
소비자 2 시작
소비자 2 대기
생산자 시작
생산자 자원 생산 후 모든 소비자에게 알림
소비자 2 자원 소비
소비자 1 자원 소비


## 프로파일링 

In [18]:
import cProfile
import time

def sleep():
    time.sleep(5)
    
def hello_world():
    print("hello world")
    
def main():
    sleep()
    hello_world()
    hello_world() #ncall이 2가 됌, 호출 횟수
cProfile.run('main()')


hello world
hello world
         70 function calls in 5.001 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.000    0.000    5.001    5.001 <ipython-input-18-cfc61f3599cb>:10(main)
        1    0.000    0.000    5.000    5.000 <ipython-input-18-cfc61f3599cb>:4(sleep)
        2    0.000    0.000    0.001    0.000 <ipython-input-18-cfc61f3599cb>:7(hello_world)
        1    0.000    0.000    5.001    5.001 <string>:1(<module>)
        5    0.000    0.000    0.001    0.000 iostream.py:197(schedule)
        4    0.000    0.000    0.000    0.000 iostream.py:309(_is_master_process)
        4    0.000    0.000    0.000    0.000 iostream.py:322(_schedule_flush)
        4    0.000    0.000    0.001    0.000 iostream.py:384(write)
        5    0.000    0.000    0.000    0.000 iostream.py:93(_event_pipe)
        5    0.001    0.000    0.001    0.000 socket.py:342(send)
        5    0.000    0.000    0.000    0.000 threadi

## Stack 만들기 

In [19]:
class Stack(object):
    def __init__(self):
        self.items = []
        
    def isEmpty(self):
        return not bool(self.items)
    
    def push(self, value):
        self.items.append(value)
        
    def pop(self):
        value = self.items.pop()
        if value is not None:
            return value
        else:
            print("Stack is empty")
            
    def size(self):
        return len(self.items)
    
    def peek(self):
        if self.items:
            return self.items[-1]
        else:
            print("Stack is empty")
            
    def __repr__(self):
        return repr(self.items)
    
if __name__ == "__main__":
    stack = Stack()
    print("스택이 비었나요? {0}".format(stack.isEmpty()))
    print("스택에 숫자 0~9를 추가합니다.")
    for i in range(10):
        stack.push(i)
    print("스택 크기 : {0}".format(stack.size()))
    print("peek: {0}".format(stack.peek()))
    print("pop: {0}".format(stack.pop()))
    print("peek: {0}".format(stack.peek()))
    print("스택이 비었나요? {0}".format(stack.isEmpty()))
    print(stack)

스택이 비었나요? True
스택에 숫자 0~9를 추가합니다.
스택 크기 : 10
peek: 9
pop: 9
peek: 8
스택이 비었나요? False
[0, 1, 2, 3, 4, 5, 6, 7, 8]


In [22]:
class Node(object):
    def __init__(self, value = None, pointer = None):
        self.value = value
        self.pointer = pointer
        
class Stack(object):
    def __init__(self):
        self.head = None
        self.count = 0
    
    def isEmpty(self):
        return not bool(self.head)
    
    def push(self, item):
        self.head = Node(item, self.head)
        self.count += 1
        
    def pop(self):
        if(self.count > 0 and self.head ):
            node = self.head
            self.head = node.pointer
            self.count -= 1
            return node.value
        else:
            print("Stack is empty")
            
    def size(self):
        return self.count
    
    def peek(self):
        if self.count > 0 and self.head:
            return self.head.value
        else:
            print("Stack is empty")
            
    def _printlist(self):
        node = self.head
        while node:
            print(node.value, end = ' ')
            node = node.pointer
        print()
    
    
if __name__ == "__main__":
    stack = Stack()
    print("스택이 비었나요? {0}".format(stack.isEmpty()))
    print("스택에 숫자 0~9를 추가합니다.")
    for i in range(10):
        stack.push(i)
    print("스택 크기 : {0}".format(stack.size()))
    print("peek: {0}".format(stack.peek()))
    print("pop: {0}".format(stack.pop()))
    print("peek: {0}".format(stack.peek()))
    print("스택이 비었나요? {0}".format(stack.isEmpty()))
    stack._printlist()

스택이 비었나요? True
스택에 숫자 0~9를 추가합니다.
스택 크기 : 10
peek: 9
pop: 9
peek: 8
스택이 비었나요? False
8 7 6 5 4 3 2 1 0 


## Queue 

In [27]:
class Queue(object):
    def __init__(self):
        self.items = []
        
    def isEmpty(self):
        return not bool(self.items)
    
    def enqueue(self, value):
        self.items.insert(0,value) #insert는 첫번쨰 파라미터 앞에 삽입
        
    def dequeue(self):
        value = self.items.pop()
        if value is not None:
            return value
        else:
            print("Queue is empty")
            
    def size(self):
        return len(self.items)
    
    def peek(self):
        if self.items:
            return self.items[-1]
        else:
            print("Queue is empty")
            
    def __repr__(self):
        return repr(self.items)
    
if __name__ == "__main__":
    queue = Queue()
    print("큐가 비었나요? {0}".format(queue.isEmpty()))
    print("큐에 숫자 0~9를 추가합니다.")
    for i in range(10):
        queue.enqueue(i)
    print("큐 크기 : {0}".format(queue.size()))
    print("peek: {0}".format(queue.peek()))
    print("pop: {0}".format(queue.dequeue()))
    print("peek: {0}".format(queue.peek()))
    print("큐가 비었나요? {0}".format(stack.isEmpty()))
    print(queue)

큐가 비었나요? True
큐에 숫자 0~9를 추가합니다.
큐 크기 : 10
peek: 0
pop: 0
peek: 1
큐가 비었나요? False
[9, 8, 7, 6, 5, 4, 3, 2, 1]


In [29]:
class Node(object):
    def __init__(self, value = None, pointer = None):
        self.value = value
        self.pointer = pointer
        
class LinkedQueue(object):
    def __init__(self):
        self.head = None
        self.tail = None
        self.count = 0
    
    def isEmpty(self):
        return not bool(self.head)
    
    def enqueue(self, value):
        node = Node(value)
        if not self.head:
            self.head = node
            self.tail = node
        else:
            if self.tail:
                self.tail.pointer = node
            self.tail = node
        self.count +=1
        
    def dequeue(self):
        if self.head:
            value = self.head.value
            self.head = self.head.pointer
            self.count -= 1
            return value
        else:
            print("Queue is empty.")
            
    def size(self):
        return self.count
    
    def peek(self):
        if self.count > 0:
            return self.head.value
        else:
            print("Queue is empty")
            
    def _printlist(self):
        node = self.head
        while node:
            print(node.value, end = ' ')
            node = node.pointer
        print()
    
    
if __name__ == "__main__":
    queue = LinkedQueue()
    print("스택이 비었나요? {0}".format(queue.isEmpty()))
    print("스택에 숫자 0~9를 추가합니다.")
    for i in range(10):
        queue.enqueue(i)
    print("스택 크기 : {0}".format(queue.size()))
    print("peek: {0}".format(queue.peek()))
    print("pop: {0}".format(queue.dequeue()))
    print("peek: {0}".format(queue.peek()))
    print("스택이 비었나요? {0}".format(queue.isEmpty()))
    queue._printlist()

스택이 비었나요? True
스택에 숫자 0~9를 추가합니다.
스택 크기 : 10
peek: 0
pop: 0
peek: 1
스택이 비었나요? False
1 2 3 4 5 6 7 8 9 


## heapq 모듈 

In [31]:
import heapq
list1 = [4,5,8,1]
heapq.heapify(list1)
list1

[1, 4, 8, 5]

In [32]:
h = []
heapq.heappush(h,(1, 'food'))
heapq.heappush(h,(2, 'have fun'))
heapq.heappush(h,(3, 'work'))
heapq.heappush(h,(4, 'study'))
h

[(1, 'food'), (2, 'have fun'), (3, 'work'), (4, 'study')]

## heap 구현 

In [44]:
class Heapify(object):
    def __init__(self,data= None):
        self.data = data or []
        for i in range(len(data)//2, -1, -1):
            self.__max_heapify__(i)
            
    def parent(self, i):
        if i & 1:
            return i >> 1
        else:
            return (i >> 1) -1
    
    def left_child(self,i):
        return (i << 1) + 1 # = i*2 +1
    
    def right_child(self,i):
        return (i<<1) + 2 # = i * 2  + 2
    
    def __max_heapify__(self,i):
        largest = i
        left = self.left_child(i)
        right = self.right_child(i)
        n = len(self.data)
        
        #왼쪽 자식
        largest = (left < n and self.data[left] > self.data[i]) and left or i# -> 무조건 false
        #오른쪽 자식
        largest = (right < n and self.data[right] > self.data[i]) and self.data[largest] and right or largest
        
        if i is not largest:
            self.data[i], self.data[largest] = self.data[largest], self.data[i]
            print(self.data)
            self.__max_heapify__(largest)
        
    def extract_max(self):
        n = len(self.data)
        max_element = self.data[0]
        self.data[0] = self.data[n-1]
        self.data = self.data[:n-1]
        self.__max_heapify__(0)
        return max_element
    
    def insert(self, item):
        i = len(self.data)
        self.data.append(item)
        while( i != 0) and item > self.data[self.parent(i)]:
            self.data[i] = self.data[self.parent(i)]
            i = self.parent(i)
        self.data[i] = item
        
        
def test_heapify():
    l1 = [3,2,5,1,7,8,2]
    h = Heapify(l1)
    print(h.extract_max())
    print(h.extract_max())
    print(h.extract_max())
    h.insert(9)
    print(h.data)

if __name__ == "__main__":
    test_heapify()

[3, 2, 8, 1, 7, 5, 2]
[3, 7, 8, 1, 2, 5, 2]
[8, 7, 3, 1, 2, 5, 2]
[8, 7, 5, 1, 2, 3, 2]
[5, 7, 2, 1, 2, 3]
[5, 7, 3, 1, 2, 2]
8
[3, 7, 2, 1, 2]
5
[7, 2, 2, 1]
3
[9, 7, 2, 1, 2]
